# Image generation using GAN

## Imports

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
import cv2
import os
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Rescaling

## Memory growth for GPU

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

## Load Images and augment data

Path to the dataset used and setting the image size and batch size:

In [ ]:
img_height, img_width = 200, 200
batch_size = 32
train_datagen  = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
    data_format="channels_last",
    brightness_range=[0.5, 1.5],
)

validation_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
img_dir = "./../data/images/classes"  # Enter Directory of all images
train_generator = train_datagen.flow_from_directory(
    "./../data/images/classes",          # Directory path to the training dataset
    target_size=(img_height, img_width),   # Resize images to 150x150 pixels
    batch_size=32,            # Number of images to yield per batch
    class_mode='categorical'  # Use 'categorical' for multi-class classification
)

validation_generator = validation_datagen.flow_from_directory(
    "./../data/images/classes",     # Directory path to the validation dataset
    target_size=(img_height, img_width),   # Resize images to 150x150 pixels
    batch_size=32,            # Number of images to yield per batch
    class_mode='categorical'  # Use 'categorical' for multi-class classification
)

In [ ]:
num_classes = 13

## Test data augmentation on image classification

In [ ]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Conv2D(
            32, (3, 3), activation="relu", input_shape=(img_height, img_width, 3)
        ),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.Dense(
            num_classes, activation="softmax"
        ),  # 13 output neurons for 13 classes
    ]
)
model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"],
)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=30,  # Number of epochs to train
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
)


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Example image path for prediction
img_path = './../data/images/classes/dangerous/6-dangerous.png'

# Load and preprocess the image
img = load_img(img_path, target_size=(img_height, img_width))
img_array = img_to_array(img)
img_array = img_array / 255.0  # Rescale to [0, 1]
img_array = tf.expand_dims(img_array, 0)  # Expand dimensions to create batch size of 1

In [ ]:
predictions = model.predict(img_array)

In [ ]:
predicted_class_index = np.argmax(predictions[0])

In [ ]:
class_labels = list(train_generator.class_indices.keys())  # Assuming train_generator is used for training
predicted_class_label = class_labels[predicted_class_index]

print("Predicted class:", predicted_class_label)

## GAN network